# Special members

> **NOTE**:
>
> * This notebook lesson requires a proper understanding of ***dynamically allocated memory*** (In Chapter III - Compound data types, notebook 4)
> * This lesson won't require be execution of anything, so there will be no code to be executed between, as the examples

Special member functions are member functions that are implicitly defined as members of classes under certain circumstances. There are six:

<p align="center">
<table class="boxed"><tbody><tr><th>Member function</th><th>typical form for class <code>C</code>:</th></tr>
<tr><td><a href="#default_constructor">Default constructor</a></td><td><code>C::C();</code></td></tr>
<tr><td><a href="#destructor">Destructor</a></td><td><code>C::~C();</code></td></tr>
<tr><td><a href="#copy_constructor">Copy constructor</a></td><td><code>C::C (const C&amp;);</code></td></tr>
<tr><td><a href="#copy_assignment">Copy assignment</a></td><td><code>C&amp; operator= (const C&amp;);</code></td></tr>
<tr><td><a href="#move">Move constructor</a></td><td><code>C::C (C&amp;&amp;);</code></td></tr>
<tr><td><a href="#move">Move assignment</a></td><td><code>C&amp; operator= (C&amp;&amp;);</code></td></tr>
</tbody></table>
</p>

The six special members' functions are members implicitly declared on classes under certain circumstances:

<p align="center">
<table class="boxed"><tbody><tr><th>Member function</th><th>implicitly defined:</th><th>default definition:</th></tr>
<tr><td><a href="#default_constructor">Default constructor</a></td><td>if no other constructors</td><td>does nothing</td></tr>
<tr><td><a href="#destructor">Destructor</a></td><td>if no destructor</td><td>does nothing</td></tr>
<tr><td><a href="#copy_constructor">Copy constructor</a></td><td>if no move constructor and no move assignment</td><td>copies all members</td></tr>
<tr><td><a href="#copy_assignment">Copy assignment</a></td><td>if no move constructor and no move assignment</td><td>copies all members</td></tr>
<tr><td><a href="#move">Move constructor</a></td><td>if no destructor, no copy constructor, and no copy nor move assignment</td><td>moves all members</td></tr>
<tr><td><a href="#move">Move assignment</a></td><td>if no destructor, no copy constructor, and no copy nor move assignment</td><td>moves all members</td></tr>
</tbody></table>
</p>

Notice how not all special member functions are implicitly defined in the same cases. This is mostly due to backward compatibility with C structures and earlier C++ versions, and some include deprecated cases. Fortunately, each class can select explicitly which of these members exist with their default definition or which are deleted by using the keywords default and delete, respectively.

Note that, the keyword default does not define a member function equal to the default constructor (i.e., where default constructor means constructor with no parameters), but equal to the constructor that would be implicitly defined if not deleted.

In general, and for future compatibility, classes that explicitly define one copy/move constructor or one copy/move assignment but not both, are encouraged to specify either delete or default on the other special member functions they don't explicitly define.


## Default constructor

The default constructor is the constructor called when objects of a class are declared but are not initialized with any arguments.

If a class definition has no constructors, the compiler assumes the class to have an implicitly defined default constructor. Therefore, after declaring a class like this:

```c++
class Example {
  public:
    int total;
    void accumulate (int x) { total += x; }
};
```
The compiler assumes that Example has a default constructor. Therefore, objects of this class can be constructed by simply declaring them without any arguments:

```c++
Example ex;
```

But as soon as a class has some constructor taking any number of parameters explicitly declared, the compiler no longer provides an implicit default constructor, and no longer allows the declaration of new objects of that class without arguments. For example, the following class:

```c++
class Example2 {
  public:
    int total;
    Example2 (int initial_value) : total(initial_value) { };
    void accumulate (int x) { total += x; };
};
```

Here, we have declared a constructor with a parameter of type int. Therefore the following object declaration would be correct:

```c++
Example2 ex (100);   // ok: calls constructor
```

## Destructor

Destructors fulfill the opposite functionality of constructors: They are responsible for the necessary cleanup needed by a class when its lifetime ends. The classes we have defined in previous chapters did not allocate any resource and thus did not really require any clean up.

But now, let's imagine that the class in the last example allocates dynamic memory to store the string it had as data member; in this case, it would be very useful to have a function called automatically at the end of the object's life in charge of releasing this memory. To do this, we use a destructor. A destructor is a member function very similar to a default constructor: it takes no arguments and returns nothing, not even void. It also uses the class name as its own name, but preceded with a tilde sign (~)"

## Copy constructor

When an object is passed a named object of its own type as argument, its copy constructor is invoked in order to construct a copy.

A copy constructor is a constructor whose first parameter is of type reference to the class itself (possibly const qualified) and which can be invoked with a single argument of this type. For example, for a class MyClass, the copy constructor may have the following signature:

```c++ 
MyClass::MyClass (const MyClass&);
```

If a class has no custom copy nor move constructors (or assignments) defined, an implicit copy constructor is provided. This copy constructor simply performs a copy of its own members. 

An implicit copy constructor is automatically defined. The definition assumed for this function performs a shallow copy, roughly equivalent to:

```c++
MyClass::MyClass(const MyClass& x) : a(x.a), b(x.b), c(x.c) {}
```

This default copy constructor may suit the needs of many classes. But shallow copies only copy the members of the class themselves, and this is probably not what we expect for classes like class Example4 we defined above, because it contains pointers of which it handles its storage. For that class, performing a shallow copy means that the pointer value is copied, but not the content itself; This means that both objects (the copy and the original) would be sharing a single string object (they would both be pointing to the same object), and at some point (on destruction) both objects would try to delete the same block of memory, probably causing the program to crash on runtime. 

The deep copy performed by this copy constructor allocates storage for a new string, which is initialized to contain a copy of the original object. In this way, both objects (copy and original) have distinct copies of the content stored in different locations.


## Copy assignment

The copy assignment operator is an overload of operator= which takes a value or reference of the class itself as parameter. The return value is generally a reference to *this (although this is not required). For example, for a class MyClass, the copy assignment may have the following signature:

```c++
MyClass& operator= (const MyClass&);
```

The copy assignment operator is also a special function and is also defined implicitly if a class has no custom copy nor move assignments (nor move constructor) defined.


## Move constructor and assignment

Similar to copying, moving also uses the value of an object to set the value to another object. But, unlike copying, the content is actually transferred from one object (the source) to the other (the destination): the source loses that content, which is taken over by the destination. This moving only happens when the source of the value is an unnamed object.

Unnamed objects are objects that are temporary in nature, and thus haven't even been given a name. Typical examples of unnamed objects are return values of functions or type-casts.

The move constructor and move assignment are members that take a parameter of type rvalue reference to the class itself:

```c++
MyClass (MyClass&&);             // move-constructor
MyClass& operator= (MyClass&&);  // move-assignment 
```

An rvalue reference is specified by following the type with two ampersands (&&). As a parameter, an rvalue reference matches arguments of temporaries of this type.

The concept of moving is most useful for objects that manage the storage they use, such as objects that allocate storage with new and delete. In such objects, copying and moving are really different operations:
- Copying from A to B means that new memory is allocated to B and then the entire content of A is copied to this new memory allocated for B.
- Moving from A to B means that the memory already allocated to A is transferred to B without allocating any new storage. It involves simply copying the pointer.

Compilers already optimize many cases that formally require a move-construction call in what is known as Return Value Optimization. Most notably, when the value returned by a function is used to initialize an object. In these cases, the move constructor may actually never get called.

Note that even though rvalue references can be used for the type of any function parameter, it is seldom useful for uses other than the move constructor. Rvalue references are tricky, and unnecessary uses may be the source of errors quite difficult to track.
